<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-06 13:46:19
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.35 C
-------------------
Today PnL: -92.56 K (-0.68%)
Current PnL: -19.82 L (-13.48%)
CY Booked + Current PnL: -11.55 L (-7.85%)
-------------------
Total profit:  2.17 L
Total loss:  -21.99 L
-------------------
Total Booked + Current PnL: 15.43 L (12.86%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.12%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 78.08 L (57.82%)
Deployed:  1.20 C
Current:  1.35 C
CAGR/XIRR %: 7.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,761.55,-19.90,H-LC,5.30,110128.0,5816.0,11718.0,0.17,5.58,10.64,16.81,10.0,0.50,0.84,22.52,X5K,ATH,METALS
67,SBIN,760.30,863.00,-11.51,M-LC,3.99,214252.0,12772.0,14441.0,0.99,6.34,6.74,13.51,62.0,0.88,1.63,19.74,XY25,NTT,BANKS
82,TTKPRESTIG,769.29,770.00,-9.35,H-SC,11.65,82792.0,-17985.0,18082.0,0.54,-17.85,21.84,0.09,145.0,-0.99,0.63,8.77,OX40N,NTT,DURABLES
71,SIS,477.00,477.00,2241.45,H-SC,7.00,62994.0,-18096.0,18098.0,-0.96,-22.32,28.73,0.00,157.0,-1.00,0.48,27.67,OX40N,NTT,MISC
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.46,154083.0,27883.0,18290.0,0.11,22.09,11.87,36.59,156.0,1.52,1.17,37.38,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,NATIONALUM,189.63,247.44,-50.15,H-MC,2.88,99677.0,-827.0,31468.0,0.43,-0.82,31.57,30.49,81.0,-0.03,0.76,31.32,MH,ATH,METALS
91,WIPRO,243.46,420.00,-12.39,M-LC,5.89,149798.0,-1147.0,110596.0,-1.75,-0.76,73.83,72.51,55.0,-0.01,1.14,5.22,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.84,H-LC,1.80,203363.0,-1137.0,42544.0,-1.03,-0.56,20.92,20.25,9.0,-0.03,1.55,8.13,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-14.41,H-LC,2.64,169891.0,-31495.0,119111.0,2.53,-15.64,70.11,43.51,19.0,-0.26,1.29,18.26,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-5.66,H-LC,3.24,181872.0,-18438.0,78423.0,-1.55,-9.20,43.12,29.94,24.0,-0.24,1.38,25.65,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.42,H-LC,3.75,185976.0,-14530.0,63306.0,-0.73,-7.25,34.04,24.33,30.0,-0.23,1.41,7.24,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-17.42,H-LC,4.19,261846.0,-3580.0,60486.0,-1.29,-1.35,23.10,21.44,26.0,-0.06,1.99,5.20,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,2961.56,4250.00,-14.41,H-LC,2.64,169891.0,-31495.0,119111.0,2.53,-15.64,70.11,43.51,19.0,-0.26,1.29,18.26,X40,BTT,PAINTS
84,UNIONBANK,123.87,163.00,-15.25,M-LC,15.61,148981.0,8141.0,36351.0,2.38,5.78,24.40,31.59,72.0,0.22,1.13,33.80,XY24,NTT,BANKS
2,ACC,2579.05,3906.00,-38.07,M-MC,4.88,133415.0,-54856.0,151720.0,1.43,-29.14,113.72,51.45,178.0,-0.36,1.01,2.19,XY24,BTT,CEMENT
54,MASFIN,326.60,399.50,-19.15,H-SC,16.34,91860.0,-6120.0,27990.0,1.37,-6.25,30.47,22.32,151.0,-0.22,0.70,33.23,XR,ATH,FINANCE
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.39,133465.0,10129.0,123629.0,1.26,8.21,92.63,108.45,118.0,0.08,1.01,32.40,AR,ATH,MISC


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BRITANNIA,5502.66,6362.67,8.33,H-LC,9.30,194490.0,-3606.0,34561.0,-4.13,-1.82,17.77,15.63,31.0,-0.10,1.48,19.63,XY25,ATH,FMCG
60,RAJESHEXPO,517.65,518.00,1663.89,L-SC,3.42,48137.0,-89040.0,89130.0,-3.95,-64.91,185.16,0.07,266.0,-1.00,0.37,19.72,OX40N,NTT,JEWELLERY
75,SYMPHONY,1306.42,1306.00,-39.43,M-SC,9.43,129192.0,-41949.0,41897.0,-3.33,-24.51,32.43,-0.03,197.0,-1.00,0.98,0.00,OX40N,NTT,DURABLES
74,SURYODAY,189.78,240.00,40.35,H-SC,5.05,99184.0,-50742.0,90416.0,-3.23,-33.84,91.16,26.46,142.0,-0.56,0.75,27.06,XR,NTT,BANKS
49,KANSAINER,299.63,340.00,-69.48,H-SC,1.67,211914.0,-57753.0,94090.0,-2.77,-21.42,44.40,13.47,136.0,-0.61,1.61,7.18,XY24,NTT,PAINTS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,128.79,H-SC,4.67,150164.0,-24395.0,24432.0,0.50,-13.97,16.27,0.02,135.0,-1.00,1.14,30.22,OX40N,NTT,PAINTS
82,TTKPRESTIG,769.29,770.0,-9.35,H-SC,11.65,82792.0,-17985.0,18082.0,0.54,-17.85,21.84,0.09,145.0,-0.99,0.63,8.77,OX40N,NTT,DURABLES
83,UJJIVANSFB,52.77,53.0,34.19,H-SC,20.65,116721.0,-25758.0,26379.0,-1.73,-18.08,22.60,0.44,163.0,-0.98,0.89,37.76,OX40N,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-21.85,H-MC,2.05,81970.0,-20682.0,20681.0,-1.79,-20.15,25.23,-0.00,101.0,-1.00,0.62,24.60,OX40N,NTT,IT
20,CERA,7989.07,8422.0,-20.76,H-SC,2.84,113139.0,-30664.0,38456.0,-1.87,-21.32,33.99,5.42,155.0,-0.80,0.86,24.85,OX40N,NTT,CERAMICS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.38,H-LC,14.87,237042.0,-58892.0,152797.0,-0.81,-19.90,64.46,31.73,1.0,-0.39,1.80,1.20,X40,BTT,IT
88,VBL,492.64,671.64,-11.22,H-LC,11.34,269457.0,4417.0,91885.0,-1.43,1.67,34.10,36.33,3.0,0.05,2.05,15.19,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-19.60,H-LC,8.67,258082.0,-3519.0,149146.0,-1.27,-1.35,57.79,55.67,4.0,-0.02,1.96,4.68,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.54,H-LC,8.75,197925.0,-18594.0,111491.0,-0.18,-8.59,56.33,42.91,5.0,-0.17,1.50,3.46,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,0.97,H-LC,9.08,178901.0,-11745.0,58250.0,-1.10,-6.16,32.56,24.40,8.0,-0.20,1.36,8.79,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,20.18,H-SC,13.46,154083.0,27883.0,18290.0,0.11,22.09,11.87,36.59,156.0,1.52,1.17,37.38,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.50,H-SC,13.50,132413.0,7116.0,51959.0,-1.32,5.68,39.24,47.15,144.0,0.14,1.01,65.66,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.46,154083.0,27883.0,18290.0,0.11,22.09,11.87,36.59,156.0,1.52,1.17,37.38,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-2.38,H-MC,4.92,157132.0,6219.0,51084.0,-1.26,4.12,32.51,37.97,90.0,0.12,1.19,30.47,AR,ATH,AUTO
19,CAMS,3643.00,5226.82,-4.58,H-SC,1.77,107094.0,5090.0,39261.0,0.74,4.99,36.66,43.48,120.0,0.13,0.81,22.61,X40N,ATH,MISC
25,DABUR,505.20,735.00,-7.18,H-MC,4.97,201182.0,4154.0,85462.0,-1.69,2.11,42.48,45.49,100.0,0.05,1.53,15.42,XY24,BTT,FMCG


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.50,H-SC,13.50,132413.0,7116.0,51959.0,-1.32,5.68,39.24,47.15,144.0,0.14,1.01,65.66,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,-2.38,H-MC,4.92,157132.0,6219.0,51084.0,-1.26,4.12,32.51,37.97,90.0,0.12,1.19,30.47,AR,ATH,AUTO
90,WHIRLPOOL,1219.98,2270.00,-41.21,M-SC,5.71,98850.0,7352.0,71399.0,0.06,8.03,72.23,86.07,211.0,0.10,0.75,43.00,XR,NTT,DURABLES
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.39,133465.0,10129.0,123629.0,1.26,8.21,92.63,108.45,118.0,0.08,1.01,32.40,AR,ATH,MISC
91,WIPRO,243.46,420.00,-12.39,M-LC,5.89,149798.0,-1147.0,110596.0,-1.75,-0.76,73.83,72.51,55.0,-0.01,1.14,5.22,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-55.73,L-SC,21.61,66528.0,-47021.0,87072.0,-1.65,-41.41,130.88,35.27,267.0,-0.54,0.51,62.83,XR,NTT,HOTELS
54,MASFIN,326.60,399.50,-19.15,H-SC,16.34,91860.0,-6120.0,27990.0,1.37,-6.25,30.47,22.32,151.0,-0.22,0.70,33.23,XR,ATH,FINANCE
3,ALKYLAMINE,2347.98,4546.37,-0.30,H-SC,18.79,92803.0,-8160.0,102696.0,-2.52,-8.08,110.66,93.63,149.0,-0.08,0.71,38.23,SR,ATH,CHEMICALS
87,VALIANTORG,512.64,838.00,-322.71,H-SC,19.80,94796.0,-38490.0,123083.0,0.00,-28.88,129.84,63.47,147.0,-0.31,0.72,58.90,XR,NTT,CHEMICALS
36,HINDALCO,651.95,761.55,-19.90,H-LC,5.30,110128.0,5816.0,11718.0,0.17,5.58,10.64,16.81,10.0,0.50,0.84,22.52,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-23.54,H-SC,7.01,217688.0,24668.0,23380.0,0.40,12.78,10.74,24.89,128.0,1.06,1.66,39.11,X40N,NTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.46,154083.0,27883.0,18290.0,0.11,22.09,11.87,36.59,156.0,1.52,1.17,37.38,XY24,NTT,HEALTHCARE
69,SHALBY,261.39,327.00,941.47,M-SC,8.41,141718.0,-39425.0,84889.0,-1.13,-21.76,59.90,25.10,233.0,-0.46,1.08,13.57,XY24,NTT,HEALTHCARE
88,VBL,492.64,671.64,-11.22,H-LC,11.34,269457.0,4417.0,91885.0,-1.43,1.67,34.10,36.33,3.0,0.05,2.05,15.19,X40N,ATH,FMCG
27,DIXON,14367.00,18940.15,-27.84,H-MC,6.63,132800.0,17864.0,18725.0,-1.48,15.54,14.10,31.83,85.0,0.95,1.01,49.46,X40N,ATH,IT


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.69
1,25,41.49
2,50,71.40


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.67
LC    37.14
MC    22.19
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.98
X40N     15.17
X40      13.28
XY25     11.70
XR        8.99
OX40N     7.68
AR        7.67
X5K       2.23
MH        1.77
X200      1.38
SR        1.15
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.98
H-SC    27.35
H-MC    19.21
M-SC    11.89
M-LC     7.10
M-MC     2.63
L-SC     1.43
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.23,-2.96,36.05
IT,11.99,-18.13,75.51
FINANCE,8.35,-18.77,61.86
BANKS,7.52,-15.23,62.24
PAINTS,5.79,-14.52,37.09
HEALTHCARE,5.11,2.25,22.43
MISC,5.10,-21.67,92.86
ELECTRICAL,4.70,-13.66,62.88
AUTO,4.39,-16.92,67.78


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2827761.0,23
XR,1110262.0,13
AR,1053985.0,9
X40,825891.0,10
X40N,612208.0,12
XY25,509405.0,8
OX40N,367868.0,10
SR,192833.0,2
X5K,146375.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2725291.0,30
M-SC,1407623.0,15
H-LC,1342000.0,20
H-MC,1178871.0,15
M-LC,444604.0,5
M-MC,331831.0,2
L-SC,274540.0,3
L-MC,56368.0,1
L-LC,47310.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      948972.0      7
M-SC       XY24      860083.0      7
H-SC       AR        610895.0      4
H-LC       X40       548760.0      6
H-SC       XR        517650.0      6
H-MC       XY24      460429.0      4
M-MC       XY24      331831.0      2
H-LC       X40N      312519.0      6
           AR        233566.0      2
H-SC       X40N      216305.0      4
H-MC       XY25      210501.0      2
           X40       201462.0      3
H-SC       SR        192833.0      2
           OX40N     186677.0      6
L-SC       XR        185410.0      2
M-LC       XY24      184910.0      2
M-SC       AR        158440.0      2
M-LC       X5K       134657.0      1
M-SC       XY25      121675.0      1
           XR        120376.0      2
H-MC       XR        119862.0      1
H-LC       XY25      115478.0      3
M-LC       XR        110596.0      1
L-SC       OX40N      89130.0      1
H-MC       X40N       83384.0      2
H-LC       X200       78423.0      1
M-SC       X40        75669.0      1
           OX40N      71380.0      2
L-MC       XR         56368.0      1
H-SC       MH         51959.0      1
H-MC       AR         51084.0      1
L-LC       XY25       47310.0      1
H-LC       XY24       41536.0      1
H-MC       MH         31468.0      1
           OX40N      20681.0      1
M-LC       XY25       14441.0      1
H-LC       X5K        11718.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
